# Shane's Swing Trading Strategy

* Stocks have support levels
    * e.g. MSFT hits recent low of $400 on May 1st, then hits a similar low of $390 on Aug 5th
    * Stocks reaching support levels is a buying opportunity
* However, good stocks trend up
    * This means the support level of good stocks rise over time
    * Thus it's difficult to rely on historical price data to determine support levels
* To adjust for this, we calculate support levels not by historical price but rather historical difference between price and 200 SMA
    * 200 SMA is a good indicator for the general direction of a stock
    * The difference between the price and the 200 SMA then measures the residual of the price from its expected price

In [ ]:
import alpaca_trade_api as alpaca
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, LimitOrderRequest, TakeProfitRequest, StopLossRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderClass
from dotenv import load_dotenv
import os
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import time
import datetime as dt

In [ ]:
paper = True

# initialize API from API keys in .env
load_dotenv()

if paper:
    api_key = os.environ['APCA-API-PAPER-KEY-ID']
    api_secret_key = os.environ['APCA-API-PAPER-SECRET-KEY']
    api_base_url = 'https://paper-api.alpaca.markets'
else:
    api_key = os.environ['APCA-API-KEY-ID']
    api_secret_key = os.environ['APCA-API-SECRET-KEY']
    api_base_url = 'https://api.alpaca.markets'

api = alpaca.REST(api_key, api_secret_key, api_base_url)
account = api.get_account()
trading_client = TradingClient(api_key, api_secret_key, paper=paper)

In [ ]:
trading_client.get_all_positions()

In [ ]:
def market_buy_order(ticker, quantity):
    time.sleep(0.2)
    market_order_data = MarketOrderRequest(
                        symbol=ticker,
                        qty=quantity,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.DAY
                        )
    market_order = trading_client.submit_order(order_data=market_order_data)
    print(market_order)
    
def market_sell_order(ticker, quantity):
    time.sleep(0.2)
    market_order_data = MarketOrderRequest(
                        symbol=ticker,
                        qty=quantity,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.DAY
                        )
    market_order = trading_client.submit_order(order_data=market_order_data)
    print(market_order)

def centered_bracket_sell_order(ticker, quantity, price, radius):
    time.sleep(0.2)
    limit_order = TakeProfitRequest(
        limit_price=round(price * (1 + radius), 2)
    )
    stop_loss = StopLossRequest(
        stop_price=round(price * (1 - radius), 2)
    )
    limit_order_data = LimitOrderRequest(
        symbol=ticker,
        qty=quantity,
        side=OrderSide.SELL,
        type="limit",
        time_in_force=TimeInForce.GTC,
        order_class = 'oco',
        take_profit=limit_order,
        stop_loss=stop_loss
    )
    market_order = trading_client.submit_order(order_data=limit_order_data)
    print(market_order)

In [ ]:
def get_SP500():
    return pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

def add_business_days(start_date: dt.date, n: int) -> dt.date:
    weeks, extra_days = divmod(n, 5)
    result_date = start_date + dt.timedelta(weeks=weeks)

    if extra_days != 0:
        start_weekday = result_date.weekday()
        
        if start_weekday + extra_days > 4:
            extra_days += 2
        result_date += dt.timedelta(days=extra_days)
    return result_date

def bars_last_n_days(ticker, days):
    now = dt.datetime.now(dt.timezone.utc)
    todays_date = now.strftime("%Y-%m-%d")
    start_date = (now - dt.timedelta(days=days)).strftime("%Y-%m-%d")
    fh_date_time = start_date + "T14:30:00Z"
    bars = api.get_bars(ticker, alpaca.TimeFrame.Day, fh_date_time, adjustment='split').df
    return bars

def bars_last_n_weeks(ticker, weeks):
    now = dt.datetime.now(dt.timezone.utc)
    todays_date = now.strftime("%Y-%m-%d")
    start_date = (now - dt.timedelta(weeks=weeks)).strftime("%Y-%m-%d")
    fh_date_time = start_date + "T14:30:00Z"
    query_attempts = 0
    while (True):
        query_attempts += 1
        if query_attempts >= 3:
            raise Exception('query failed')
        try:
            bars = api.get_bars(ticker, alpaca.TimeFrame.Day, fh_date_time, adjustment='split').df
            break
        except:
            time.sleep(3)
    bars = bars.reset_index()
    bars = bars.rename(columns={'timestamp': 'date'})
    bars['date'] = pd.to_datetime(bars['date']).dt.date
    bars = bars.set_index('date')
    return bars

def add_SMA(bars, days):
    bars[f"{days} day SMA"] = ta.SMA(bars['close'], timeperiod=days)
    bars[f"close - {days} day SMA"] = bars['close'] - bars[f"{days} day SMA"]
    return bars[bars[f"{days} day SMA"].notna()]

def SMA_slope(bars, sma_days):
    time_period = 5
    return (bars.iloc[-1][f"{sma_days} day SMA"] - bars.iloc[-1 - time_period][f"{sma_days} day SMA"]) / time_period

def get_troughs(bars, n_troughs, radius):
    trough_dates = []
    sorted_bars = bars.sort_values('close - 200 day SMA')
    for i in range(len(sorted_bars)):
        close_prox = False
        row = sorted_bars.iloc[i]
        date = row.name
        if add_business_days(date, 2) >= dt.datetime.now(dt.timezone.utc).date():
            continue
        for tdate in trough_dates:
            lower_bound = add_business_days(tdate, -radius)
            upper_bound = add_business_days(tdate, radius)
            if date >= lower_bound and date <= upper_bound:
                close_prox = True
                break
        if close_prox:
            continue
        trough_dates.append(date)
        if len(trough_dates) >= n_troughs:
            break
    return sorted_bars[sorted_bars.index.isin(trough_dates)].sort_index()

def latest_date(bars):
    latest = bars.iloc[0].name
    for i in range(len(bars)):
        date = bars.iloc[i].name
        if date > latest:
            latest = date
    return latest

def get_industry_holdings():
    industry_holdings = {}
    SP500 = get_SP500()
    all_positions = trading_client.get_all_positions()
    for position in all_positions:
        ticker = position.symbol
        industry = SP500.loc[SP500['Symbol'] == ticker].iloc[0]['GICS Sub-Industry']
        industry_holdings[industry] = 1 + industry_holdings.get(industry, 0)
    return industry_holdings


def recently_sold_for_loss(ticker):
    blacklist_days = 50

    orders = api.list_orders(symbols=[ticker], side='sell', status='closed', limit=100)
    for order in orders:
        if order.type == 'stop':
            if order.status == 'filled':
                if dt.datetime.now(dt.timezone.utc) - order.canceled_at < dt.timedelta(days=blacklist_days):
                    return True
    return False

In [ ]:
def execute_long_trading_strategy(paper=True, simulation=False):
    troughs_std_threshold = 0.031 # 0.025
    SMA_slope_yearly_threshold = 0.20
    buy_amount = 3000
    price_lower_bound = 5
    price_upper_bound = 1500
    profit_taking_radius = 0.03
    minimum_trough_threshold = -0.04
    below_minimum_trough_threshold = 0.02

    if not paper:
        buy_amount = 1500
        price_upper_bound = 800

    blacklist = {'FCX'}

    passed_std_threshold = []
    passed_current_price_threshold = []
    passed_slope_threshold = []
    passed_both_slope_std_threshold = []

    held_tickers = []
    all_positions = trading_client.get_all_positions()
    for position in all_positions:
        held_tickers.append(position.symbol)
    industry_holdings = get_industry_holdings()

    SP500 = get_SP500()
    # tickers = pd.read_csv('trading_tickers.csv')
    
    for ticker in SP500['Symbol']:
    # for ticker in ['GE', 'LDOS', 'STX', 'NVDA', 'PGR', 'JPM', 'MSFT', 'GOOG', 'AAPL', 'FIS', 'ECL', 'FANG']:
    # for ticker in ['HPQ']:
        time.sleep(0.05)
        bars = bars_last_n_weeks(ticker, 68) # 85
        print(ticker)
        bars = add_SMA(bars, 200)
        # display(bars)
        if bars.empty:
            continue
        troughs = get_troughs(bars, 3, 13) # 7 5
        # display(troughs)
        # latest_trough_date = latest_date(troughs)
        # troughs_range = troughs['close - 200 day SMA'].max() - troughs['close - 200 day SMA'].min()
        troughs_std = troughs['close - 200 day SMA'].std()
        troughs_mean = troughs['close - 200 day SMA'].mean()
        troughs_min = troughs['close - 200 day SMA'].min()
        latest_trade = api.get_latest_trade(ticker)
        current_price = latest_trade.p
        # print(f"std: {troughs_std / current_price}")
        # print(f"mean: {troughs_mean / current_price}")
        # print(f"lowest: {troughs['close - 200 day SMA'].min() / current_price}")
        last_sma = bars.iloc[-1]['200 day SMA']
        SMA_200_slope_percentage = SMA_slope(bars, 200) / current_price

        # print(f"current_price: {current_price}")
        # print(f"troughs_std: {troughs_std / current_price}")
        # print(f"troughs_mean: {troughs_mean}")
        # print(f"last sma 200: {last_sma}")
        # print(f"SMA 200 slope percentage yearly: {SMA_200_slope_percentage * 365}")

        industry = SP500.loc[SP500['Symbol'] == ticker].iloc[0]['GICS Sub-Industry']

        # if latest_trough_date + dt.timedelta(days=trough_recency_threshold) < dt.datetime.now(dt.timezone.utc).date():
        #     print('not recent')
            # continue

        if troughs_std / current_price <= troughs_std_threshold:
            passed_std_threshold.append(ticker)
        if current_price - last_sma <= troughs_mean:
            passed_current_price_threshold.append(ticker)
        if SMA_200_slope_percentage * 365 >= SMA_slope_yearly_threshold:
            passed_slope_threshold.append(ticker)
        if troughs_std / current_price <= troughs_std_threshold and \
            SMA_200_slope_percentage * 365 >= SMA_slope_yearly_threshold:
            passed_both_slope_std_threshold.append(ticker)

        if current_price < price_lower_bound:
            continue
        if current_price > price_upper_bound:
            continue
        if troughs_std / current_price > troughs_std_threshold:
            continue 
        if current_price - last_sma > troughs_mean:
            continue
        if SMA_200_slope_percentage * 365 < SMA_slope_yearly_threshold:
            continue
        if troughs_min < current_price * minimum_trough_threshold:
            print("minimum trough too low")
            continue
        if current_price * (1 + below_minimum_trough_threshold) - last_sma < troughs_min:
            print("fell too far below minimum trough")
            continue
        if ticker in held_tickers:
            print("repeat ticker")
            continue
        if recently_sold_for_loss(ticker):
            print("recently traded for loss")
            continue
        if industry_holdings.get(industry, 0) >= 2:
            print("already have >2 holdings in same industry")
            continue
        if float(account.cash) < buy_amount * 1.5:
            print("not enough cash")
            continue
        if ticker in blacklist:
            print("blacklisted")
            continue
        shares_to_buy = round(buy_amount / current_price)
        if not simulation:
            market_buy_order(ticker, shares_to_buy)
        time.sleep(2)
        if not simulation:
            centered_bracket_sell_order(ticker, shares_to_buy, current_price, profit_taking_radius)
        print(f"bought {ticker}")
    print(f"passed std threshold: {len(passed_std_threshold)}")
    print(passed_std_threshold)
    print(f"passed slope threshold: {len(passed_slope_threshold)}")
    print(passed_slope_threshold)
    print(f"passed both: {len(passed_both_slope_std_threshold)}")
    print(passed_both_slope_std_threshold)
    print(f"passed current price threshold: {len(passed_current_price_threshold)}")
    print(passed_current_price_threshold)

In [ ]:
execute_long_trading_strategy(paper=paper, simulation=True)

In [ ]:
print(account)

In [ ]:
ticker = 'META'
bars = bars_last_n_weeks(ticker, 85)
bars = add_SMA(bars, 200)
print(SMA_slope(bars, 200))
display(bars)
bars.sort_values('close - 200 day SMA').head(20)

In [ ]:
print(SMA_slope(bars, 200))
slope = SMA_slope(bars, 200)
latest_trade = api.get_latest_trade(ticker)
current_price = latest_trade.p
print(slope / current_price)
print(slope / current_price * 365)

In [ ]:
plt.plot(bars['close - 200 day SMA'])
plt.show()
plt.clf()

In [ ]:
troughs = get_troughs(bars, 7, 5)
troughs_range = troughs['close - 200 day SMA'].max() - troughs['close - 200 day SMA'].min()
troughs_std = troughs['close - 200 day SMA'].std()
troughs_mean = troughs['close - 200 day SMA'].mean()
print(troughs_range)
print(troughs_std)
print(troughs_mean)
print(bars.iloc[-1]['200 day SMA'])
troughs